In [1]:
import pandas as pd
import numpy as np

data_path = '../input/nlp-pre-processing'
X_train = np.load(f'{data_path}/X_train.npy')
y_train = np.load(f'{data_path}/y_train.npy')
X_test = np.load(f'{data_path}/X_test.npy')

In [2]:
from sklearn.model_selection import train_test_split
from keras import Sequential, layers, callbacks

# Xtr, Xval, ytr, yval = train_test_split(X_train, y_train, test_size=0.2)
MODELS = dict()
HISTORY = dict()

Using TensorFlow backend.


## Spacy Doc2Vec
Actually just the mean of word vectors in a doc oops

In [3]:
model = Sequential([
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(58, activation='softmax'),
])

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
)

MODELS['spacy_nlp'] = model

In [4]:
for name, model in MODELS.items():
    print('MODEL:', name)
    HISTORY[name] = model.fit(
        X_train, y_train,
        epochs = 100,
        batch_size = 128,
        validation_split = 0.2,
        callbacks = [
            callbacks.EarlyStopping(patience=5), 
            callbacks.ModelCheckpoint(name+'_best_weights.hdf5', save_weights_only=True, save_best_only=True)
            ]
        )

MODEL: spacy_nlp
Train on 533292 samples, validate on 133323 samples
Epoch 1/100
533292/533292 [==============================] - 23s 42us/step - loss: 1.3838 - acc: 0.5881 - val_loss: 1.3052 - val_acc: 0.6600
Epoch 2/100
533292/533292 [==============================] - 21s 40us/step - loss: 1.2329 - acc: 0.6238 - val_loss: 1.1639 - val_acc: 0.6937
Epoch 3/100
533292/533292 [==============================] - 21s 40us/step - loss: 1.1923 - acc: 0.6342 - val_loss: 1.1094 - val_acc: 0.6979
Epoch 4/100
533292/533292 [==============================] - 22s 40us/step - loss: 1.1659 - acc: 0.6414 - val_loss: 1.1399 - val_acc: 0.7002
Epoch 5/100
533292/533292 [==============================] - 21s 40us/step - loss: 1.1481 - acc: 0.6457 - val_loss: 1.0842 - val_acc: 0.7071
Epoch 6/100
533292/533292 [==============================] - 21s 40us/step - loss: 1.1337 - acc: 0.6485 - val_loss: 1.1112 - val_acc: 0.6920
Epoch 7/100
533292/533292 [==============================] - 21s 40us/step - loss: 1.

In [5]:
best_model = MODELS['spacy_nlp']
best_model.load_weights('spacy_nlp_best_weights.hdf5')

In [6]:
!cp ../input/ndsc-utils/ndsc_utils.py ndsc_utils.py
from ndsc_utils import Extractor

train = pd.read_csv('../input/ndsc-beginner/test.csv')
itemid = Extractor()(train, 'itemid')
categories = best_model.predict_classes(X_test)

predictions = pd.DataFrame({
    'itemid': itemid,
    'Category': categories,
})

predictions.to_csv('nlp_transfer_learning.csv', index=False)